In [ ]:
# Script that receives an unclassified PSF dataset and adds the remaining stars in the FOV with SEDs assigned by a CNN classifier

import numpy as np
import tensorflow as tf
import sklearn.metrics as skm

In [ ]:
# Load datasets, there is the assumption that the datasets are coherent
dataset_path = '/Users/as274094/Documents/psf_dataset4/'
dataset_name_1 = 'train_Euclid_500_stars_id_004GT_350_bins.npy'
dataset_name_2 = 'train_Euclid_2000_stars_id_004GT_350_bins.npy'
dataset_1 = np.load(dataset_path + dataset_name_1, allow_pickle=True)[()] # Dataset with true SEDs
dataset_2 = np.load(dataset_path + dataset_name_2, allow_pickle=True)[()] # The dataset containing the whole FOV

In [ ]:
n_remaining_stars = dataset_2['noisy_stars'].shape[0]-dataset_1['noisy_stars'].shape[0]

true_SEDs = np.array(dataset_2['SED_ids'][-n_remaining_stars:])
x_to_convert = np.expand_dims(dataset_2['noisy_stars'][-n_remaining_stars:], axis = 3)
true_SEDs.shape

In [ ]:
# Load model

model_path = '/Users/as274094/GitHub/Refractored_star_classifier/tensorflow_version/best_models/CNN_model/'
classifier = tf.keras.models.load_model(model_path)
    
def calculate_success_rate(confusion_matrix):
    diagonal = np.trace(confusion_matrix)
    diagonal_neighbors = np.sum(np.diagonal(confusion_matrix, offset=1)) + np.sum(np.diagonal(confusion_matrix, offset=-1))
    total_classified = np.sum(confusion_matrix)
    
    success_rate = (diagonal + diagonal_neighbors) / total_classified
    return success_rate


In [ ]:
# Make predictions and calculate metrics

y_test_pred = classifier.predict(x_to_convert, verbose = 1)
class_predictions = np.argmax(y_test_pred, axis = 1)

f1_mean = np.mean(skm.f1_score(true_SEDs, class_predictions, average = None)[:13])
print('Average F1 score:', f1_mean)

confusion_matrix = skm.confusion_matrix(true_SEDs, class_predictions)
print("\nConfusion matrix:")
print(confusion_matrix)

success_rate = calculate_success_rate(confusion_matrix)
print('\nSuccess rate:', success_rate)

In [ ]:
# Assign SEDs
concatenated_SEDs = np.load('concatenated_SEDs.npy', allow_pickle=True)[()]

SED_list = []
for spectral_class in class_predictions:
    concat_SED = concatenated_SEDs[spectral_class]
    SED_list.append(concat_SED)
SED_array = np.array(SED_list, dtype=object)
SED_array.shape

In [ ]:
# Save the new dataset

dataset_2['SED_ids'] = np.append(dataset_1['SED_ids'],class_predictions)
dataset_2['SEDs'] = np.concatenate((dataset_1['SEDs'],SED_array))
dataset_2['F1'] = f1_mean
dataset_2['success_rate'] = success_rate

np.save(
        dataset_path + 'expanded_CNN_' + dataset_name_1,
        dataset_2,
        allow_pickle=True
    )

In [ ]:
# Verification

expanded_dataset = np.load('/Users/as274094/Documents/psf_dataset4/expanded_CNN_train_Euclid_1000_stars_id_004GT_350_bins.npy', allow_pickle=True)[()]
full_dataset = np.load('/Users/as274094/Documents/psf_dataset4/train_Euclid_2000_stars_id_004GT_350_bins.npy', allow_pickle=True)[()]
print(expanded_dataset['F1'],expanded_dataset['success_rate'])

In [ ]:
SEDs_full = full_dataset['SED_ids'][1000:]
SEDs_expanded = expanded_dataset['SED_ids'][1000:]

f1_mean = np.mean(skm.f1_score(SEDs_full, SEDs_expanded, average = None)[:13])
print('Average F1 score:', f1_mean)

confusion_matrix = skm.confusion_matrix(SEDs_full, SEDs_expanded)
print("\nConfusion matrix:")
print(confusion_matrix)

success_rate = calculate_success_rate(confusion_matrix)
print('\nSuccess rate:', success_rate)